In [2]:
import pandas as pd
import numpy as np
import re
from sympy import symbols

Function2

In [3]:
def salary_growing_by_year(state, occ_code, display=0):
    df = pd.read_csv('./filtered_state_M2022_dl.csv')
    result = df.loc[(df['AREA_TITLE'] == state) & (df['OCC_CODE'] == occ_code)]
    A_PCT10 = int(result['A_PCT10'].values[0])
    A_PCT25 = int(result['A_PCT25'].values[0])
    A_MEDIAN = int(result['A_MEDIAN'].values[0])
    if result['A_PCT75'].values[0] == '#':
#         print('##')
        A_PCT75 = 2*A_MEDIAN - A_PCT25
    else:
        A_PCT75 = int(result['A_PCT75'].values[0])

    if result['A_PCT90'].values[0] == '#':
#         print('##')
        A_PCT90 = 2*A_PCT75 - A_MEDIAN
    else:
        A_PCT90 = int(result['A_PCT90'].values[0])
    # A_PCT90
    x = np.array([0, 5, 10, 15, 20])
    y = np.array([A_PCT10, A_PCT25, A_MEDIAN, A_PCT75, A_PCT90])
    
    coeff = np.polyfit(x, y, 3)
    
    coeff = np.polyfit(x, y, 3)

    if display :
        # Generate values for the curve
        x_curve = np.linspace(0, 20, 100)
        poly_function = np.poly1d(coeff)
        y_curve = poly_function(x_curve)

        # Plotting the original data points and the fitted curve
        plt.figure(figsize=(8, 6))
        plt.scatter(x, y, label='Original Data Points')
        plt.plot(x_curve, y_curve, color='red', label='Fitted Curve')
        plt.xlabel('Years')
        plt.ylabel('Salaries')
        plt.title(f'Fitted Polynomial Curve for')
        plt.legend()
        plt.grid(True)
        plt.show()

    return coeff

In [4]:
# Function to find the tax rate
def find_tax_rate(state, income):
    
    new_file_path = 'filtered_income_tax_2022_manual.csv'  # Replace with your file path
    data = pd.read_csv(new_file_path)
    """
    Find the tax rate based on state and income.

    Args:
    state (str): The state name.
    income (float): The income amount.
    data (DataFrame): The DataFrame with tax information.

    Returns:
    float: The corresponding tax rate.
    """
    # Filter data for the specific state
    state_data = data[data['State'].str.contains(state, case=False, na=False)]

    # Sort by Bracket to ensure correct order
    state_data = state_data.sort_values(by='Bracket')

    # Initialize default rate
    rate = None

    # Find the correct bracket for the given income
    for index, row in state_data.iterrows():
        if income > row['Bracket']:
            rate = row['Rates']
        else:
            break

    return rate

### function3 estimate the year to pay off the house mortgage

In [5]:
def estimate_payoff_house_mortgage(occupation, state) :
    # mapping to the occ_code
    occupation_mapping = {
        'Management Occupations': '11-0000',
        'Business and Financial Operations Occupations': '13-0000',
        'Computer and Mathematical Occupations': '15-0000',
        'Architecture and Engineering Occupations': '17-0000',
        'Life, Physical, and Social Science Occupations': '19-0000',
        'Community and Social Service Occupations': '21-0000',
        'Legal Occupations': '23-0000',
        'Educational Instruction and Library Occupations': '25-0000',
        'Arts, Design, Entertainment, Sports, and Media Occupations': '27-0000',
        'Healthcare Practitioners and Technical Occupations': '29-0000',
        'Healthcare Support Occupations': '31-0000',
        'Protective Service Occupations': '33-0000',
        'Food Preparation and Serving Related Occupations': '35-0000',
        'Building and Grounds Cleaning and Maintenance Occupations': '37-0000',
        'Personal Care and Service Occupations': '39-0000',
        'Sales and Related Occupations': '41-0000',
        'Office and Administrative Support Occupations': '43-0000',
        'Farming, Fishing, and Forestry Occupations': '45-0000',
        'Construction and Extraction Occupations': '47-0000',
        'Installation, Maintenance, and Repair Occupations': '49-0000',
        'Production Occupations': '51-0000',
        'Transportation and Material Moving Occupations': '53-0000'
    }
    occ_code = occupation_mapping.get(occupation)
    
    
    # get the salary growing function
    coeff = salary_growing_by_year(state, occ_code, display=0)
    #build the function
    # Define the variables
    y = symbols('y')  # Year
    AI = coeff[0]*y**3 + coeff[1]*y**2 + coeff[2]*y + coeff[3]  # Annual income function (third-order polynomial)
    
    #some constants
    PIM = 0.25  # Percentage of Income for Mortgage (25%)
    MHP_percent = 0.95  # Mortgage Percentage of Housing Price (95%)
    FIR = 0.005  # Federal Interest Rate (0.5%)
    
    # find the real estate tax rate based on state
    df = pd.read_csv('real_estate_tax.csv')
    find_row = df.loc[(df['State'] == state)]
    tax = find_row['Real Estate Tax Rate'].values[0]
    RET = float(tax)/100  # Real estate tax rate (3%)
#     print(RET)
    #find state house price
    df = pd.read_csv('filtered_housing_prices.csv')
    find_row = df.loc[(df['State'] == state)]
    price = find_row['Mean of Median of housing price'].values[0]
    MHP = float(price)# Median Housing Price
    
#     print(MHP)
    
    # Function to calculate annual mortgage payment for a given year
    def annual_mortgage_payment(year):
        income = AI.subs(y, year)
#         print(income)
        # find the income tax
        state_income_tax = find_tax_rate(state, income)
        federal_income_tax = find_tax_rate('Federal', income)
        IT = float(state_income_tax) +  float(federal_income_tax)
        
        after_tax_income = income * (1 - IT)  # Income after income tax
        after_all_taxes_income = after_tax_income - (MHP * RET)  # Income after all taxes
        return after_all_taxes_income * PIM
    
    initial_mortgage = MHP * MHP_percent
    # Calculating the number of years to pay off the mortgage
    remaining_mortgage = initial_mortgage
    year = 1
    while remaining_mortgage > 0:
        payment = annual_mortgage_payment(year)
        remaining_mortgage = remaining_mortgage + (remaining_mortgage * FIR) - payment
        year += 1

    return year - 1    
    

In [6]:
state = 'California'
occupation = 'Computer and Mathematical Occupations'

print(estimate_payoff_house_mortgage(occupation, state))

29


In [7]:
# from sympy import symbols



# # Define the variables
# y = symbols('y')  # Year
# AI = y**3 + 2*y**2 + 3*y + 4  # Annual income function (third-order polynomial)

# # Constants
# RET = 0.03  # Real estate tax rate (3%)
# PIM = 0.25  # Percentage of Income for Mortgage (25%)
# MHP = 300000  # Median Housing Price
# MHP_percent = 0.95  # Mortgage Percentage of Housing Price (95%)
# IT = 0.20  # Income Tax rate (20%)
# FIR = 0.005  # Federal Interest Rate (0.5%)

# # Initial Mortgage Amount
# initial_mortgage = MHP * MHP_percent

# # Function to calculate annual mortgage payment for a given year
# def annual_mortgage_payment(year):
#     income = AI.subs(y, year)
#     after_tax_income = income * (1 - IT)  # Income after income tax
#     after_all_taxes_income = after_tax_income - (MHP * RET)  # Income after all taxes
#     return after_all_taxes_income * PIM

# # Calculating the number of years to pay off the mortgage
# remaining_mortgage = initial_mortgage
# year = 1
# while remaining_mortgage > 0:
#     payment = annual_mortgage_payment(year)
#     remaining_mortgage = remaining_mortgage + (remaining_mortgage * FIR) - payment
#     year += 1

# year - 1  # Subtract 1 to get the final year of payment
